In [1]:
from vizdoom import DoomGame  
import cv2
import os
from stable_baselines3.common.callbacks import BaseCallback
from stable_baselines3 import PPO
from stable_baselines3.common import policies

In [2]:
import EnvironmentConfigurations as EnvConfig

In [3]:
from VizdoomGymWrapper import VizDoomGym

In [4]:
class AgentCallback(BaseCallback):
  def __init__(self, check_freq, save_path, verbose=1):
    super(AgentCallback, self).__init__(verbose)
    self.check_freq = check_freq
    self.save_path = save_path

  def __init_callback(self):
    if self.save_path is not None:
      os.makedirs(self.save_path, exist_ok=True)
  
  def _on_step(self):
    if self.n_calls % self.check_freq == 0:
      model_path = os.path.join(self.save_path, f"model_{self.n_calls}")
      self.model.save(model_path)
    return True
  
  
agentCallback = AgentCallback(check_freq=25000, save_path=f"{EnvConfig.AGENT_MODEL_PATH_PREFIX}{EnvConfig.configurations[EnvConfig.CURRENT_CONFIGURATION_INDEX]['name']}")

In [7]:

env = VizDoomGym(envConfig=EnvConfig.configurations[EnvConfig.CURRENT_CONFIGURATION_INDEX], render=False)  
# model = PPO('CnnPolicy', env, device='cuda', tensorboard_log=f"{TENSORBOARD_LOG_PATH_PREFIX}{configurations[CURRENT_CONFIGURATION_INDEX]['name']}", verbose=1, learning_rate=0.00001, n_steps=8192, clip_range=.1, gamma=.95, gae_lambda=.9)
model = PPO(policies.ActorCriticCnnPolicy, env, device='cuda', tensorboard_log=f"{EnvConfig.TENSORBOARD_LOG_PATH_PREFIX}{EnvConfig.configurations[EnvConfig.CURRENT_CONFIGURATION_INDEX]['name']}", verbose=1, n_epochs=34, n_steps= 4096, learning_rate= 1e-4, batch_size= 64)

model.learn(total_timesteps=3000000, callback=agentCallback)

deathmatch


In [9]:
env = VizDoomGym(configurations[CURRENT_CONFIGURATION_INDEX], render=False)
## train_env = make_vec_env(env_id=env, n_envs=4, vec_env_cls=SubprocVecEnv)

In [10]:

model = PPO.load(f"{AGENT_MODEL_PATH_PREFIX}{configurations[CURRENT_CONFIGURATION_INDEX]['name']}/model_125000")

In [11]:
model.set_env(env)
model.learn(total_timesteps=3000000, callback=agentCallback, reset_num_timesteps=False)

Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Wrapping the env in a VecTransposeImage.
Logging to ./logs/logs_for_defend_the_center_expanded\PPO_3
---------------------------------
| rollout/           |          |
|    ep_len_mean     | 249      |
|    ep_rew_mean     | 764      |
| time/              |          |
|    fps             | 22       |
|    iterations      | 1        |
|    time_elapsed    | 180      |
|    total_timesteps | 129096   |
---------------------------------
----------------------------------------
| rollout/                |            |
|    ep_len_mean          | 263        |
|    ep_rew_mean          | 791        |
| time/                   |            |
|    fps                  | 12         |
|    iterations           | 2          |
|    time_elapsed         | 681        |
|    total_timesteps      | 133192     |
| train/                  |            |
|    approx_kl            | 0.13859926 |
|    clip_fraction        | 0.3

KeyboardInterrupt: 

In [8]:
env.close()